In [1]:
import pandas as pd
import numpy as np
import os
import Memory_Collaborative_Filtering as mem
import sqlite3 as sql
import sklearn 
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import pickle

In [2]:
worm_path = 'C:/Users/arjun/Desktop/amazon.db'
freq_path = 'C:/Users/arjun/Desktop/frequency_rankings.db'
meta_path = 'C:/users/arjun/Desktop/test.csv'

In [4]:
review_data = mem.import_standard_data(worm_path)

In [5]:
top_product_counts = mem.import_product_frequency_data(freq_path)

In [6]:
product_metadata = pd.read_csv(meta_path)

In [25]:
wrangled_review_data = review_data.drop_duplicates(['customer_id', 'product_id'])
wrangled_review_data['star_rating'] = wrangled_review_data['star_rating'].astype(int)
wrangled_review_data['review_length'] = wrangled_review_data['review_body'].str.count(' ') + 1
wrangled_review_data = wrangled_review_data.drop(['customer_id'], axis=1)
wrangled_review_data = wrangled_review_data.drop_duplicates()
wrangled_review_data = wrangled_review_data.reset_index(drop=True)

C:\Users\arjun\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\arjun\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
final_product_data = pd.merge(wrangled_review_data, top_product_counts, left_on='product_id', right_on='product_id', how='inner')
final_product_data = pd.merge(final_product_data, product_metadata, left_on='product_id', right_on='product_id', how='left')
final_product_data = final_product_data[['product_id', 'star_rating', 'helpful_votes', 'review_length', 'counts', 'price', 'salesRank']]

In [27]:
final_product_data = final_product_data.dropna()
final_product_data = final_product_data[['star_rating', 'helpful_votes', 'review_length', 'counts', 'price', 'salesRank']]

In [28]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm = 'ball_tree').fit(final_product_data)

In [29]:
def get_index_from_name(name, review_data):
    return review_data[review_data["product_title"]==name].index.tolist()[0]

In [30]:
def get_id_from_partial_name(partial):
    for name in all_anime_names:
        if partial in name:
            print(name,all_anime_names.index(name))

In [31]:
def print_similar_books(review_data, query=None,id=None, model=None):
    distances2, indices2 = model.kneighbors()
    if id:
        for id in indices2[id][1:]:
            print(review_data.iloc[id]["product_title"])
    if query:
        found_id = get_index_from_name(query, review_data)
        for id in indices2[found_id][1:]:
            print(review_data.iloc[id]["product_title"])

In [32]:
test_recs = wrangled_review_data.head(5)

   product_id                                      product_title  \
0  080072433X                            Trial Run (Fault Lines)   
1  043985654X          How Do Dinosaurs Play with Their Friends?   
2  075700394X  Sodium Bicarbonate: Nature's Unique First Aid ...   
3  0399142789  Conversations with God: An Uncommon Dialogue, ...   
4  1844839737  The Big Book of Juices: More Than 400 Natural ...   

                                         review_body  star_rating  \
0  This book I actually had a hard time with. I p...            3   
1                       nice book for young children            5   
2                                        Excellent--            5   
3  LOVE this book. Read before in the 90's and wa...            5   
4                                               good            5   

   helpful_votes  review_length  
0            0.0            277  
1            0.0              5  
2            0.0              1  
3            0.0             25  
4     

In [33]:
for book in test_recs['product_title']:
    print(book + " recommendations:")
    print_similar_books(wrangled_review_data, query=book, model=nbrs)

Trial Run (Fault Lines) recommendations:
The Pioneer Woman Cooks: Food from My Frontier
Kafka on the Shore
American Born Chinese
Doctor Sleep
Frozen Little Golden Book (Disney Frozen)
How Do Dinosaurs Play with Their Friends? recommendations:
Doctor Sleep: A Novel
How to Babysit a Grandma
Manwhore +1 (The Manwhore Series)
Dead Doctors Don't Lie
Pegasus: A Novel
Sodium Bicarbonate: Nature's Unique First Aid Remedy recommendations:
Quiet: The Power of Introverts in a World That Can't Stop Talking
The Dream Lover: A Novel
American Born Chinese
My Struggle: Book 1
Alive: Book One of the Generations Trilogy
Conversations with God: An Uncommon Dialogue, Book 1 recommendations:
Missoula: Rape and the Justice System in a College Town
Dark Places
A Shade Of Vampire
Wonder
Curious George Goes Camping
The Big Book of Juices: More Than 400 Natural Blends for Health and Vitality Every Day recommendations:
Alive: Book One of the Generations Trilogy
I'll Give You the Sun
Practice to Deceive
All the B